In [45]:
!pip install neo4j

     |████████████████████████████████| 157 kB 3.2 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for neo4j: filename=neo4j-5.3.0-py3-none-any.whl size=221840 sha256=1b96fcf1ab7faa24ecde4a799934ce3c217c445b66389587d25d2eba4504f669
  Stored in directory: /Users/meghashishodia/Library/Caches/pip/wheels/2a/e0/1d/86615be499a1516f755f4df1f9489cfd3bad5c0e77129c2bee
Successfully built neo4j


In [39]:
#only certain versions of JAVA are supported by Neo4j
%env JAVA_HOME = /usr/local/Cellar/openjdk@17/17.0.5/libexec/openjdk.jdk/Contents/Home

env: JAVA_HOME=/usr/local/Cellar/openjdk@17/17.0.5/libexec/openjdk.jdk/Contents/Home


In [1]:
import csv
import json
import pandas as pd
import sys, getopt, pprint
import neo4j
from neo4j import GraphDatabase

In [2]:
#Install Neo4j and start Neo4j server using neo4j/bin start

In [70]:
#connecting to Neo4j
driver = GraphDatabase.driver("bolt://localhost:7687")
session = driver.session(database="neo4j")

In [67]:
#coverting the first dataset from .dat to .csv to be consumed by Neo4j for persistence
import pandas as pd

data = pd.read_csv("movielens/movies.dat", sep='::', skipinitialspace=True,
                   names=['Movie_Id','Movie_Title','Genres'])

data.to_csv('movielens/movies.csv', header=True, index=False)
data.head()

<ipython-input-67-f1b6b7abdf9d>:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv("/Users/meghashishodia/532/movies.dat", sep='::', skipinitialspace=True,


,Movie_Id,Movie_Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [68]:
#coverting the second dataset from .dat to .csv to be consumed by Neo4j for persistence
import pandas as pd

data = pd.read_csv("movielens/users.dat", sep='::', skipinitialspace=True,
                   names=['id','Gender','Age', 'Occupation', 'Zip_Code'])

data.to_csv('movielens/users.csv', header=True, index=False)
data.head()

<ipython-input-68-c9905de703e4>:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv("/Users/meghashishodia/532/users.dat", sep='::', skipinitialspace=True,


,id,Gender,Age,Occupation,Zip_Code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [16]:
#coverting the third dataset from .dat to .csv to be consumed by Neo4j for persistence
import pandas as pd

data = pd.read_csv("movielens/ratings.dat", sep='::', skipinitialspace=True,
                   names=['User_Id', 'Movie_Id', 'Rating', 'Timestamp'])

data_new = data.head(25000)

data_new.to_csv('movielens/ratings.csv', header=True, index=False)
data.head()

<ipython-input-16-07aefaa9cedc>:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv("/Users/meghashishodia/532/ratings.dat", sep='::', skipinitialspace=True,


,User_Id,Movie_Id,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [61]:
#Persist users to Neo4j
def add_data(tx):
    tx.run("LOAD CSV FROM 'file:///users.csv' AS row RETURN row")

with driver.session() as session:
    session.execute_write(add_data)
driver.close()


In [52]:
#Persist ratings to Neo4j
def add_data(tx):
    tx.run("LOAD CSV FROM 'file:///ratings.csv' AS row RETURN row")

with driver.session() as session:
    session.execute_write(add_data)
driver.close()


In [53]:
#Persist movies to Neo4j
def add_data(tx):
    tx.run("LOAD CSV FROM 'file:///movies.csv' AS row RETURN row")


with driver.session() as session:
    session.execute_write(add_data)
driver.close()


In [ ]:
#Load the user data in Neo4j
def load_users(tx):
    tx.run("LOAD CSV WITH HEADERS FROM 'file:///users.csv' AS row \
            WITH toInteger(row.User_Id) AS User_Id, row.Gender AS Gender, \
            row.Age As Age, row.Occupation AS Occupation, row.Zip_Code AS Zip_Code \
            MERGE (p:Users {User_Id: User_Id}) \
            SET p.Gender = Gender, p.Age = Age \
            RETURN User_Id, Gender \
            LIMIT 5;")
    
    
with driver.session() as session:
    session.execute_write(load_users)
driver.close()

In [ ]:
#Load the movie data in Neo4j
def load_movies(tx):
    tx.run("LOAD CSV WITH HEADERS FROM 'file:///movies.csv' AS row \
            WITH toInteger(row.Movie_Id) AS Movie_Id, row.Movie_Title AS Movie_Title, row.Genres As Genres \
            MERGE (p:Movies {Movie_Id: Movie_Id}) \
            SET p.Movie_Title = Movie_Title, p.Genres = Genres \
            RETURN Movie_Id, Movie_Title, Genres \
            LIMIT 5;")
    
    
with driver.session() as session:
    session.execute_write(load_movies)
driver.close()

In [ ]:
#Load the ratings data in Neo4j
def load_ratings(tx):
    tx.run("LOAD CSV WITH HEADERS FROM 'file:///ratings.csv' AS row \
            WITH toInteger(row.User_Id) AS User_Id, toInteger(row.Movie_Id) AS Movie_Id, \
            toInteger(row.Rating) as Rating, row.Timestamp As Timestamp \
            MATCH (u:Users {User_Id: User_Id}) \
            MATCH (m:Movies {Movie_Id: Movie_Id}) \
            MERGE (u)-[rel:CONTAINS {Rating: Rating}]->(m) \
            RETURN count(rel);")
    
    
with driver.session() as session:
    session.execute_write(load_ratings)
driver.close()

In [ ]:
# create graph using GDS in Neo4j
def project_graph(tx):
    tx.run("CALL gds.graph.project( \
              'movie_ratings', \
              ['Users','Movies'], \
              { \
                CONTAINS: { \
                    orientation: 'UNDIRECTED', \
                    properties: 'Rating' \
                } \
              } \
            );"
          )
    
    
with driver.session() as session:
    session.execute_write(project_graph)
driver.close()

In [ ]:
# get embeddings for nodes using FastRP in GDS
def mutate_fastRP(tx):
    tx.run("CALL gds.fastRP.mutate('movie_ratings', \
              { \
                embeddingDimension: 8, \
                randomSeed: 42, \
                mutateProperty: 'embedding', \
                relationshipWeightProperty: 'Rating', \
                iterationWeights: [0.8, 1, 1, 1] \
              } \
            ) \
            YIELD nodePropertiesWritten;"
          )
    
    
with driver.session() as session:
    session.execute_write(mutate_fastRP)
driver.close()

In [ ]:
# Run KNN using GDS in Neo4j
def write_knn(tx):
    tx.run("CALL gds.knn.write('movie_ratings', { \
            topK: 2, \
            nodeProperties: ['embedding'], \
            randomSeed: 42, \
            concurrency: 1, \
            sampleRate: 1.0, \
            deltaThreshold: 0.0, \
            writeRelationshipType: "SIMILAR", \
            writeProperty: "score" \
            }) \
            YIELD nodesCompared, relationshipsWritten, similarityDistribution \
            RETURN nodesCompared, relationshipsWritten, similarityDistribution.mean as meanSimilarity;"
          )
    
    
with driver.session() as session:
    session.execute_write(write_knn)
driver.close()

In [ ]:
# Generating similarity matrix for users
def generate_similarity(tx):
    tx.run("MATCH (n:Users)-[r:SIMILAR]->(m:Users) \
            RETURN n.User_Id as person1, m.User_Id as person2, r.score as similarity \
            ORDER BY similarity DESCENDING, person1, person2;"
          )
      
with driver.session() as session:
    session.execute_write(generate_similarity)
driver.close()

In [ ]:
# Get Recommendation by specifying the user
def fetch_recommendation(tx):
    tx.run("MATCH (:Users {User_Id: 13})-->(m1:Movies) \
            WITH collect(m1) as movies \
            MATCH (:Users {User_Id: 148})-->(m2:Movies) \
            WHERE not m2 in movies \
            RETURN m2.Movie_Title as recommendation;"
          )
    
    
with driver.session() as session:
    session.execute_write(fetch_recommendation)
driver.close()

In [ ]:
for each in reader:
    row={}
    for field in header:
        row[field]=each[field]

    db.segment.insert(row)

env: JAVA_HOME=/Library/Java/JavaVirtualMachines/jdk-14.0.1.jdk/Contents/Home


DataFrame[User_Id: string, Movie_Title: string, Movie_Id: string, Genres: string, Gender: string, Age: string, Occupation: string, Rating_num: string, count_movieRatedTotal: string, count_userRatedTotal: string]

DataFrame[id: string, User_Id: string, Movie_Title: string, Movie_Id: string, Genres: string, Gender: string, Age: string, Occupation: string, count_movieRatedTotal: string]

DataFrame[src: string, dst: string, relationship: string]

+----+-------+--------------------+--------+--------------------+------+---+----------+---------------------+
|  id|User_Id|         Movie_Title|Movie_Id|              Genres|Gender|Age|Occupation|count_movieRatedTotal|
+----+-------+--------------------+--------+--------------------+------+---+----------+---------------------+
| 296|    296|Snow White and th...|     594|Animation|Childre...|     M| 50|         5|                  763|
| 296|    296|       Psycho (1960)|    1219|     Horror|Thriller|     M| 50|         5|                 1263|
| 829|    829|Snow White and th...|     594|Animation|Childre...|     M|  1|        19|                  763|
| 829|    829|In the Heat of th...|    1950|       Drama|Mystery|     M|  1|        19|                  348|
| 829|    829|       Psycho (1960)|    1219|     Horror|Thriller|     M|  1|        19|                 1263|
| 829|    829|   Annie Hall (1977)|    1230|      Comedy|Romance|     M|  1|        19|                 1334|
|1436|   1

+----+--------------------+------------+
| src|                 dst|relationship|
+----+--------------------+------------+
| 296|Snow White and th...|           3|
| 296|       Psycho (1960)|           5|
| 829|Snow White and th...|           3|
| 829|In the Heat of th...|           4|
| 829|       Psycho (1960)|           5|
| 829|   Annie Hall (1977)|           5|
|1436|    Elizabeth (1998)|           5|
|2069|       Psycho (1960)|           4|
|2088|Night of the Livi...|           4|
|2088|In the Heat of th...|           4|
|2088|       Psycho (1960)|           5|
|2088|   Annie Hall (1977)|           4|
|2162|       Psycho (1960)|           3|
|2294|Snow White and th...|           3|
|4032|7th Voyage of Sin...|           5|
|4821|Odd Couple II, Th...|           1|
|4937|    Elizabeth (1998)|           5|
|5645|Night of the Livi...|           4|
|5925|Snow White and th...|           3|
|5925|   Annie Hall (1977)|           4|
+----+--------------------+------------+
only showing top

+--------------------+--------+
|                  id|inDegree|
+--------------------+--------+
|In the Realm of t...|       1|
|       Gabbeh (1996)|       1|
|Monument Ave. (1998)|       1|
|Draughtsman's Con...|       1|
|    Firelight (1997)|       1|
|In the Bleak Midw...|       1|
|Adrenalin: Fear t...|       1|
|Swept from the Se...|       1|
|         Fall (1997)|       1|
|   Miss Julie (1999)|       1|
|Adventures of Elm...|       1|
|Brother, Can You ...|       1|
|  Light It Up (1999)|       1|
|Dreaming of Josep...|       1|
|Little Nemo: Adve...|       1|
|Time Regained (Le...|       1|
|  Hollow Reed (1996)|       1|
| Secret Agent (1936)|       1|
| Queens Logic (1991)|       1|
|American Dream (1...|       1|
+--------------------+--------+
only showing top 20 rows



+-----------------------------------------------------+--------+
|id                                                   |inDegree|
+-----------------------------------------------------+--------+
|American Beauty (1999)                               |464     |
|Star Wars: Episode V - The Empire Strikes Back (1980)|445     |
|Star Wars: Episode IV - A New Hope (1977)            |424     |
|Star Wars: Episode VI - Return of the Jedi (1983)    |421     |
|Terminator 2: Judgment Day (1991)                    |395     |
+-----------------------------------------------------+--------+



+-----------------------------------+--------+
|id                                 |inDegree|
+-----------------------------------+--------+
|Bitter Sugar (Azucar Amargo) (1996)|1       |
|X: The Unknown (1956)              |1       |
|In the Bleak Midwinter (1995)      |1       |
|American Dream (1990)              |1       |
|Secret Agent (1936)                |1       |
+-----------------------------------+--------+



+-----------------------------------+--------+
|id                                 |inDegree|
+-----------------------------------+--------+
|Bitter Sugar (Azucar Amargo) (1996)|1       |
|X: The Unknown (1956)              |1       |
|In the Bleak Midwinter (1995)      |1       |
|American Dream (1990)              |1       |
|Secret Agent (1936)                |1       |
+-----------------------------------+--------+



0.9791097640991211

+--------------------+-----------+
|Genres              |Watch_Count|
+--------------------+-----------+
|Drama               |502        |
|Comedy              |399        |
|Horror              |135        |
|Comedy|Romance      |122        |
|Comedy|Drama        |121        |
|Drama|Romance       |111        |
|Thriller            |80         |
|Documentary         |54         |
|Action              |53         |
|Drama|Thriller      |52         |
|Children's|Comedy   |44         |
|Action|Thriller     |43         |
|Drama|War           |35         |
|Comedy|Drama|Romance|31         |
|Animation|Children's|31         |
|Crime|Drama         |30         |
|Action|Drama        |30         |
|Horror|Thriller     |30         |
|Comedy|Horror       |30         |
|Action|Sci-Fi       |27         |
+--------------------+-----------+
only showing top 20 rows



0.47609901428222656